In [ ]:
# import necessary modules
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, classification_report 

In [ ]:
data_train=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
data_train.head()

In [ ]:
import missingno as msno
msno.matrix(data_train)

In [ ]:
msno.heatmap(data_train)

In [ ]:
corr=data_train.corr()
sns.heatmap(corr)

In [ ]:
data_train.isnull().sum()

In [ ]:
data_tr=data_train.copy()
data_tr.drop(["Alley","PoolQC","Fence","MiscFeature"], axis=1, inplace=True)
data_tr['LotFrontage'] = data_tr['LotFrontage'].fillna((data_tr['LotFrontage'].mean()))
data_tr['MasVnrArea'] = data_tr['MasVnrArea'].fillna((data_tr['MasVnrArea'].mean()))
data_tr['GarageYrBlt'] = data_tr['GarageYrBlt'].fillna((data_tr['GarageYrBlt'].mean()))
df_most_common_imputed = data_tr.apply(lambda x: x.fillna(x.value_counts().index[0]))
df_most_common_imputed.dropna(axis=0,inplace=True)
df_most_common_imputed

In [ ]:
df_most_common_imputed.isnull().sum()

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in df_most_common_imputed.columns:
    if df_most_common_imputed[column_name].dtype == object:
        df_most_common_imputed[column_name] = le.fit_transform(df_most_common_imputed[column_name])
    else:
        pass
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X = df_most_common_imputed.iloc[:,0:]  
y = df_most_common_imputed.iloc[:,-1] 

bestfeatures = SelectKBest(score_func=chi2, k=46)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(46,'Score')) 
df_most_common_imputed

In [ ]:
#df_most_common_imputed.drop(["MSZoning","Street","LandContour","Utilities","Condition1","Condition2","OverallCond","YearRemodAdd","RoofMatl","MasVnrType","ExterQual","ExterCond","Foundation","BsmtCond","BsmtFinType2","Heating","CentralAir","Electrical","FullBath","BedroomAbvGr","KitchenAbvGr","KitchenQual","Functional","FireplaceQu","GarageYrBlt","GarageQual","GarageCond","PavedDrive","YrSold","SaleType","SaleCondition"], axis=1, inplace=True)

In [ ]:
df_most_common_imputed

In [ ]:
sns.boxplot(data=df_most_common_imputed)

In [ ]:
# sns.pairplot(data_train,height=2)

In [ ]:
data_test=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
data_test.head()

In [ ]:
msno.matrix(data_test)

In [ ]:
msno.heatmap(data_test)

In [ ]:
corr=data_test.corr()
sns.heatmap(corr)

In [ ]:
data_test.isnull().sum()

In [ ]:
data_tt=data_test.copy()
data_tt.drop(["Alley","PoolQC","Fence","MiscFeature"], axis=1, inplace=True)
data_tt['LotFrontage'] = data_tt['LotFrontage'].fillna((data_tt['LotFrontage'].mean()))
data_tt['MasVnrArea'] = data_tt['MasVnrArea'].fillna((data_tt['MasVnrArea'].mean()))
data_tt['GarageYrBlt'] = data_tt['GarageYrBlt'].fillna((data_tt['GarageYrBlt'].mean()))
dt_most_common_imputed = data_tt.apply(lambda x: x.fillna(x.value_counts().index[0]))
dt_most_common_imputed.dropna(axis=0,inplace=True)
dt_most_common_imputed

In [ ]:
dt_most_common_imputed.isnull().sum()

In [ ]:
#dt_most_common_imputed.drop(["MSZoning","Street","LandContour","Utilities","Condition1","Condition2","OverallCond","YearRemodAdd","RoofMatl","MasVnrType","ExterQual","ExterCond","Foundation","BsmtCond","BsmtFinType2","Heating","CentralAir","Electrical","FullBath","BedroomAbvGr","KitchenAbvGr","KitchenQual","Functional","FireplaceQu","GarageYrBlt","GarageQual","GarageCond","PavedDrive","YrSold","SaleType","SaleCondition"], axis=1, inplace=True)

In [ ]:
dt_most_common_imputed

In [ ]:
sns.boxplot(data=dt_most_common_imputed)

In [ ]:
# sns.pairplot(dt_most_common_imputed)

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in dt_most_common_imputed.columns:
    if dt_most_common_imputed[column_name].dtype == object:
        dt_most_common_imputed[column_name] = le.fit_transform(dt_most_common_imputed[column_name])
    else:
        pass
x_train = df_most_common_imputed.iloc[:,0:-1]  
y_train = df_most_common_imputed.iloc[:,-1] 
x_test=   dt_most_common_imputed
x_train
y_train
x_test

In [ ]:
df=pd.DataFrame()
df['Id']=dt_most_common_imputed['Id']
df['SalePrice']=df_most_common_imputed['SalePrice']


In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor( learning_rate = 0.1,
                max_depth =4,objective="reg:linear",n_estimators=555)#555 BEST
xg_reg.fit(x_train,y_train)
xgbost= xg_reg.predict(x_test)
df["SalePrice"]=xgbost
df['SalePrice']=df['SalePrice'].astype('float')
df.head(50)

In [ ]:
sns.scatterplot(data =df, x = "Id", y = "SalePrice")
plt.show()

In [ ]:
df.info()

In [ ]:
df.to_csv("xg_boost.csv",index=False)